# Exploring the discrepancy in ICU admission labels

In [13]:
import pandas as pd


pd.set_option("display.max_columns", None) # show all cols
pd.set_option('display.max_colwidth', None) # show full width of showing cols
pd.set_option("display.expand_frame_repr", False) # print cols side by side as it's supposed to be

## Read data

In [1]:
import os
import pandas as pd

from pathlib import Path


dataDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240531/data/data_matrix.csv')
icuDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240119/final/icu_admission_labels.csv')
journeyDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240531/temp/SuperBugSuperEpisode-72hr-2024-02-23-copy.csv')
admissionsDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', '2023-11-10-blood-admissions.csv'))

dataDf.shape, icuDf.shape, journeyDf.shape, admissionsDf.shape

/tmp/ipykernel_827820/563950125.py:9: DtypeWarning: Columns (10,17,26,31,37,44) have mixed types. Specify dtype option on import or set low_memory=False.
  journeyDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240531/temp/SuperBugSuperEpisode-72hr-2024-02-23-copy.csv')


((116754, 194), (23174, 3), (95491, 46), (436214, 20))

## Check admission types

In [6]:
studyAdmissionsDf = admissionsDf.merge(
    dataDf,
    how='inner',
    left_on=['PATIENT_ID', 'EPISODE_ID'],
    right_on=['person_id', 'visit_occurrence_id']
)
studyAdmissionsDf

,PATIENT_ID,EPISODE_ID,PARENT_EPISODE_ID,TYPE,STATUS_DESCRIPTION,start_date,ADMIT_DATETIME,DISCHARGE_DATETIME,end_date,PRIMARY_VISIT_REASON,...,labs_Total alkaline phosphatase level_last,labs_Total protein measurement_last,labs_Globulin_last,labs_Calcium_last,labs_Phosphate measurement_last,labs_Magnesium measurement_last,deathtime_adm,death_7_day,death_14_day,death_30_day
0,18849,12790706,12790706,Emergency,Discharged,2019-09-01 04:58:40.000,NaN,NaN,2019-09-01 08:33:00.000,UTI,...,56.00,63.00,33.00,2.1625,0.8063,0.7037,NaN,0,0,0
1,18849,12790706,12790706,Emergency,Discharged,2019-09-01 04:58:40.000,NaN,NaN,2019-09-01 08:33:00.000,UTI,...,88.00,62.00,36.00,2.1400,0.7100,0.5900,NaN,0,0,0
2,18849,12790706,12790706,Emergency,Discharged,2019-09-01 04:58:40.000,NaN,NaN,2019-09-01 08:33:00.000,UTI,...,134.06,63.25,36.88,2.2079,0.9252,0.7422,NaN,0,0,0
3,18849,13476453,13476453,Emergency,Discharged,2020-04-17 14:07:57.000,NaN,NaN,2020-04-17 16:08:00.000,rash drug generalised,...,116.58,68.81,34.00,2.3254,1.1411,0.7789,NaN,0,0,0
4,18849,13569580,13569580,Emergency,Discharged,2020-05-22 10:10:23.000,NaN,NaN,2020-05-22 12:30:00.000,ETOH,...,61.00,66.00,30.00,2.0900,0.7100,0.7600,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261547,2684042,17566160,17566160,Inpatient,Discharged,2023-04-01 20:48:00.000,2023-04-01 20:48:00.000,2023-04-25 13:41:00.000,2023-04-25 13:41:00.000,GEN MED ADMIT,...,63.00,74.00,44.00,2.4200,1.5200,0.9700,NaN,0,0,0
261548,2684042,17566160,17566160,Inpatient,Discharged,2023-04-01 20:48:00.000,2023-04-01 20:48:00.000,2023-04-25 13:41:00.000,2023-04-25 13:41:00.000,GEN MED ADMIT,...,143.26,71.73,40.00,2.3004,1.3349,0.8991,NaN,0,0,0
261549,2684042,17566160,17566160,Inpatient,Discharged,2023-04-01 20:48:00.000,2023-04-01 20:48:00.000,2023-04-25 13:41:00.000,2023-04-25 13:41:00.000,GEN MED ADMIT,...,144.77,71.50,39.00,2.3700,1.1100,0.9900,NaN,0,0,0
261550,2684042,17565390,17565390,Emergency,Discharged,2023-03-31 23:17:45.000,NaN,NaN,2023-04-01 08:19:46.000,NaN,...,67.00,68.00,35.00,2.2200,0.8600,0.6800,NaN,0,0,0


In [7]:
studyAdmissionsDf.TYPE.value_counts()

TYPE
Inpatient               145440
Emergency               114641
Outpatient                 891
Pathology Referral         288
Preoperative               283
Trial                        6
Community/Ambulatory         3
Name: count, dtype: int64

In [8]:
studyAdmissionsDf.STATUS_DESCRIPTION.value_counts()

STATUS_DESCRIPTION
Discharged    259920
Cancelled       1632
Name: count, dtype: int64

In [11]:
studyAdmissionsDf.ADMIT_TYPE.value_counts()

ADMIT_TYPE
Planned Admission            124869
Other Emergency Admission     13123
Admission from ED              7401
Statistical Admission            47
Name: count, dtype: int64

In [17]:
studyAdmissionsDf.ADMITTING_WARD.value_counts().reset_index()[:20]

,ADMITTING_WARD,count
0,A-7EA - 7 East,35123
1,A-HOC - Haematology Oncology Centre,22302
2,A-APU - Admissions & Perioperative Unit,16399
3,A-ICU - Intensive Care Unit,8245
4,A-VHOC - Virtual Haematology Oncology Centre,6398
5,A-3WS - Ward 3 West,4660
6,A-3EA - Ward 3 East,4233
7,C-AC2 - Aged Care 2,4070
8,A-5EA - 5 East,3279
9,A-6EA - 6 East,3258


In [31]:
studyAdmissionsDf[studyAdmissionsDf.ADMITTING_WARD == 'A-ICU - Intensive Care Unit'][['PATIENT_ID', 'EPISODE_ID']].drop_duplicates()

,PATIENT_ID,EPISODE_ID
5271,268318,338786
5278,268473,350333
5316,284067,473733
5931,287090,524324
6789,292229,425232
...,...,...
260152,2663525,17224359
260319,2665827,17285820
260769,2674115,17395120
261327,2676719,17439008


In [32]:
studyAdmissionsDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates()

,PATIENT_ID,EPISODE_ID
0,18849,12790706
3,18849,13476453
4,18849,13569580
5,41308,14824675
8,49412,14824910
...,...,...
261499,2681377,17651112
261515,2681377,17706553
261521,2682433,17536805
261524,2684042,17566160


In [34]:
round(studyAdmissionsDf[studyAdmissionsDf.ADMITTING_WARD == 'A-ICU - Intensive Care Unit'][['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]/studyAdmissionsDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]*100, 2)

0.7

In [29]:
round(studyAdmissionsDf[studyAdmissionsDf.ADMITTING_WARD == 'A-ICU - Intensive Care Unit'].shape[0]/studyAdmissionsDf.shape[0]*100, 2)

3.15